In [211]:
def calculate_stock_roi(bb_signals_nd, adj_close_nd, periods_date, periods_list, tickers, n_sample, initial_investment, percent_to_buy, percent_to_sell):
    # Initialize a nested dictionary to store ROI percentages for each period and ticker
    roi_results = {period: {ticker: [] for ticker in tickers} for period in periods_list}

    # Loop through each economic period
    for period in periods_list:
        # Create the date range for the current period
        date_range = pd.date_range(start=pd.to_datetime(periods_date[period][0]), end=pd.to_datetime(periods_date[period][1]) - timedelta(days=90))
        
        # Get random dates for stochastic modeling
        start_dates = np.random.choice(date_range, size=n_sample, replace=True)

        # Loop through sampled start dates
        for start_date in start_dates:
            time_stamp = pd.to_datetime(start_date)

            # Initialize variables
            account_balance = initial_investment
            shares_number = {ticker: 0 for ticker in tickers}  # Initialize share count for each ticker
            shares_value = {ticker: 0 for ticker in tickers}   # Initialize share value for each ticker

            # Extract the adjusted close and signal data for time period
            adj_close_period = adj_close_nd[period].loc[time_stamp:time_stamp + timedelta(days=90)]
            bb_signals_period = bb_signals_nd[period].loc[time_stamp:time_stamp + timedelta(days=90)]

            # Iterate over each row in the Bollinger Band signals (day by day)
            for row_idx, row in bb_signals_period.iterrows():
                for col_idx, signal in enumerate(row):
                    ticker = tickers[col_idx]  # Correctly get ticker for each column
                    adj_close_price = adj_close_period.loc[row_idx, ticker]  # Get corresponding adjusted close price

                    # Handle Buy action
                    if signal == 'Buy':
                        amount_to_buy = percent_to_buy * account_balance
                        if account_balance >= amount_to_buy:
                            shares_to_buy = amount_to_buy / adj_close_price
                            shares_number[ticker] += shares_to_buy
                            account_balance -= amount_to_buy

                    # Handle Sell action
                    elif signal == 'Sell':
                        if shares_number[ticker] > 0:
                            shares_value[ticker] = shares_number[ticker] * adj_close_price
                            amount_to_sell = percent_to_sell * shares_value[ticker]
                            if shares_value[ticker] >= amount_to_sell:
                                shares_to_sell = amount_to_sell / adj_close_price
                                shares_number[ticker] -= shares_to_sell
                                account_balance += amount_to_sell

            # Calculate total portfolio value at the end of the period
            portfolio_value = sum(shares_number[ticker] * adj_close_period.iloc[-1][ticker] for ticker in tickers)
            total_value = account_balance + portfolio_value
            
            # Calculate the profit relative to the initial investment
            profit = total_value - initial_investment
            
            # Calculate ROI for each stock as a percentage of the initial investment
            for ticker in tickers:
                if shares_number[ticker] > 0:  # Only consider tickers with shares owned
                    roi_dollar_value = shares_value[ticker] - (initial_investment * (percent_to_buy * shares_number[ticker]))

                else:
                    roi_dollar_value = 0

                # Store ROI in the results dictionary
                roi_results[period][ticker].append(roi_dollar_value)

    return roi_results

In [212]:
import yfinance as yf
missing_data_tickers = [] # use this as a list of tickers with missing data

def get_data_from_start_to_end(ticker, start_date, end_date):
    global missing_data_tickers  # Use the global list to accumulate missing tickers
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if stock_data.empty:
            missing_data_tickers.append(ticker)
            raise ValueError(f"Stock data for ticker {ticker} during the period from {start_date} to {end_date} was not found.")
        return stock_data
    except Exception as e:
        print(f"An error occurred for ticker {ticker}: {e}")
        missing_data_tickers.append(ticker)
        return None


In [213]:
# for a variety of periods load in different list of tickers
def download_stock_data_for_periods(tickers, periods):
    all_data = {}
    
    for period, (start_date, end_date) in periods.items():
        period_data = {}
        for ticker in tickers:
            data = get_data_from_start_to_end(ticker, start_date, end_date)
            if data is not None:
                period_data[ticker] = data
        all_data[period] = period_data
    
    return all_data

In [214]:
import pandas as pd

# Get the adjusted close prices
adj_close_sector_etf = {}

# Create adjusted close price only listing of sector ETFs
def get_adjusted_closed_price(nested_dict, tickers, periods):
    for period in periods:
        stock_price_df = pd.DataFrame()  # Create a new DataFrame for each period
        for ticker in tickers:
            stock_price_df[ticker] = nested_dict[period][ticker]['Adj Close']
        
        adj_close_sector_etf[period] = stock_price_df  # Store the complete DataFrame for the period
    
    return adj_close_sector_etf

In [215]:
import os
import pandas as pd

# Define the root directory
root_dir = "/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Financial Capstone Project/data/signal_data"

# Initialize an empty dictionary
signals_dict = {}

# Loop through the directories and files
for foldername, subfolders, filenames in os.walk(root_dir):
    # Get the signal type from the folder name (last part of the path)
    signal_type = os.path.basename(foldername)
    
    # Loop through the files
    for filename in filenames:
        # Check if the file is a CSV
        if filename.endswith('.csv'):
            # Extract the period from the file name (e.g., "expansion", "contraction", etc.)
            period = filename.split('_')[0]  # Assuming period is the first part of the file name

            # Initialize the dictionary for the period if not already present
            if period not in signals_dict:
                signals_dict[period] = {}

            # Build the full file path
            file_path = os.path.join(foldername, filename)
            
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Add the DataFrame to the nested dictionary under the corresponding period and signal type
            signals_dict[period][signal_type] = df.set_index('Date')

In [216]:
from datetime import timedelta

# create a list of signal pages
signals_list = ['bollinger_bands','rsi_signals','ichimoku_signals','macd_signals']

# create an all_data functions
def combined_data(adj_close_data,signals_list,periods,tickers):
    # create an all data dictionary where each period is a list
    all_data = {period: {} for period in periods}
    for period in periods:
        # add to all data with different periods in different signals with each ticker as the columns
        all_data[period] = {signal: pd.DataFrame(columns=tickers) for signal in signals_list}

        # add the adjusted close sector etf data
        all_data[period]['adj_close'] = adj_close_data[period]

            # add the different signals data
        for signal in signals_list:
            all_data[period][signal] = signals_dict[period][signal]
    
    return all_data


In [217]:
# import packages that will be used for analysis
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [218]:
# create time periods for where this takes place
economic_cycle_periods = {

    "trough": ("2008-10-01", "2009-06-01"),
    "expansion": ("2012-01-01", "2015-01-01"),
    "peak": ("2019-06-01", "2020-02-01"),
    "contraction": ("2007-12-01", "2008-10-01"),
    'alldata': ('2005-01-01','2024-06-01')
}

economic_cycle_periods_list = ['trough','expansion','peak','contraction','alldata']

In [219]:
# create etf tickers for sectors
sector_etf_tickers = [
    'XLB', # materials sector
    'XLI', # industrials sector
    'XLF', # financials
    'XLK', # information technology
    'XLY', # consumer discretionary
    'XLP', # consumer staples
    'XLE', # energy
    'XLV', # healthcare
    'VOX', # communication services
    'XLU', # utilities
    'IYR' # real estate
    ]

In [220]:
# save nested dictionary data as a variable to be accessed.
sector_etf_data = download_stock_data_for_periods(sector_etf_tickers,economic_cycle_periods)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [221]:
adj_close_data = get_adjusted_closed_price(sector_etf_data,sector_etf_tickers,economic_cycle_periods_list)

In [222]:
all_data = combined_data(adj_close_sector_etf,signals_list,economic_cycle_periods_list,sector_etf_tickers)

In [285]:
import copy  # Import the copy module

columns = ['adj_close','bollinger_bands','rsi_signals','ichimoku_signals','macd_signals']



for period in economic_cycle_periods_list:  

    # create the daterange for the current period
    date_range = pd.date_range(start=pd.to_datetime(economic_cycle_periods[period][0]), end=pd.to_datetime(economic_cycle_periods[period][1]) - timedelta(days=90))

    # create a start date random from the period that is selected
    start_dates = np.random.choice(date_range,size=1,replace=True)

    start_dates = pd.to_datetime(start_dates)

    # loop through each of the samples and perform the analysis
    for start_date in start_dates:

        # for each piece of data in all_data dict get only the investment timestamp
        for col in columns:
            investment_period = copy.deepcopy(all_data)

            # Ensure the index is a datetime type
            investment_period[period][col].index = pd.to_datetime(investment_period[period][col].index)
            #print(period)
            #print(col)
            #print(investment_period[period][col].loc[start_date:start_date+timedelta(days=90)])

            # Perform the slicing for the 90-day investment period
            investment_period[period][col] = investment_period[period][col]
        for day in range(len(investment_period[period][col].loc[start_date:(start_date + timedelta(days=90))])): # move down (date)
            for ticker in sector_etf_tickers: # move across for tickers
                for signal in signals_list: # move across signals
                    row = investment_period[period][col].iloc[day] # you are at a row of 
                    date_range = pd.date_range(start=pd.to_datetime(start_date),end =start_date+timedelta(days=90))
                    print(pd.to_datetime(date_range[day].value))
                    print(period)
                    print(ticker)
                    print(signal)
                    print(day)
                    print(row[ticker])


                    
                

2008-11-01 00:00:00
trough
XLB
bollinger_bands
0
Hold
2008-11-01 00:00:00
trough
XLB
rsi_signals
0
Hold
2008-11-01 00:00:00
trough
XLB
ichimoku_signals
0
Hold
2008-11-01 00:00:00
trough
XLB
macd_signals
0
Hold
2008-11-01 00:00:00
trough
XLI
bollinger_bands
0
Hold
2008-11-01 00:00:00
trough
XLI
rsi_signals
0
Hold
2008-11-01 00:00:00
trough
XLI
ichimoku_signals
0
Hold
2008-11-01 00:00:00
trough
XLI
macd_signals
0
Hold
2008-11-01 00:00:00
trough
XLF
bollinger_bands
0
Hold
2008-11-01 00:00:00
trough
XLF
rsi_signals
0
Hold
2008-11-01 00:00:00
trough
XLF
ichimoku_signals
0
Hold
2008-11-01 00:00:00
trough
XLF
macd_signals
0
Hold
2008-11-01 00:00:00
trough
XLK
bollinger_bands
0
Hold
2008-11-01 00:00:00
trough
XLK
rsi_signals
0
Hold
2008-11-01 00:00:00
trough
XLK
ichimoku_signals
0
Hold
2008-11-01 00:00:00
trough
XLK
macd_signals
0
Hold
2008-11-01 00:00:00
trough
XLY
bollinger_bands
0
Hold
2008-11-01 00:00:00
trough
XLY
rsi_signals
0
Hold
2008-11-01 00:00:00
trough
XLY
ichimoku_signals
0
Hold
2

In [275]:
pd.to_datetime(date_range[0].value)

Timestamp('2020-09-20 00:00:00')

In [259]:
investment_period['alldata']['adj_close']

,XLB,XLI,XLF,XLK,XLY,XLP,XLE,XLV,VOX,XLU,IYR
Date,,,,,,,,,,,
2005-01-03,19.055208,21.009325,16.521315,16.185612,27.308887,13.832361,20.082979,21.374723,35.235577,13.839283,29.067562
2005-01-04,18.712055,20.750118,16.363815,15.884014,26.989534,13.784411,19.979624,21.202810,35.100529,13.743766,28.668585
2005-01-05,18.453064,20.606871,16.336651,15.829885,26.888266,13.712499,19.893509,21.166994,34.663227,13.527604,27.595903
2005-01-06,18.576084,20.675079,16.418114,15.783484,26.748060,13.778423,20.237986,21.324585,34.701813,13.597985,27.832417
2005-01-07,18.608456,20.579584,16.331224,15.814422,26.740269,13.844344,20.077229,21.310253,34.888306,13.597985,27.830038
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-24,90.522812,123.632500,41.296124,214.427994,174.802490,76.117775,89.899681,143.321762,132.972839,70.450714,83.969780
2024-05-28,90.176018,122.122604,40.839596,215.334763,174.374313,75.525421,90.824547,141.585083,133.241226,70.381744,83.456627
2024-05-29,88.887932,120.394165,40.512085,213.969620,173.229141,74.982422,89.220779,140.483521,132.535461,69.475250,82.677025


In [248]:
investment_period['alldata']['adj_close'] = investment_period['alldata']['adj_close'].loc['2024-05-01':'2024-05-31']
investment_period['alldata']['adj_close']

,XLB,XLI,XLF,XLK,XLY,XLP,XLE,XLV,VOX,XLU,IYR
Date,,,,,,,,,,,
2024-05-01,88.273605,120.414040,40.055553,193.572098,173.786774,74.064270,90.549057,139.510971,126.919136,66.519272,81.630981
2024-05-02,87.817825,121.109383,40.154800,196.362183,175.977539,74.686249,91.070526,139.411728,128.509613,66.873993,82.716499
2024-05-03,88.709579,121.953728,40.244122,201.832733,177.162552,74.913315,91.080368,139.759079,129.513580,67.406067,83.348076
2024-05-06,89.244629,123.185493,40.760197,204.204300,179.054550,74.952805,91.798630,140.324738,131.501648,67.750938,83.555305
2024-05-07,90.304825,123.523224,40.899143,203.536682,177.769989,75.772240,91.710068,141.485855,131.799881,68.519485,84.295433
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-24,90.522812,123.632500,41.296124,214.427994,174.802490,76.117775,89.899681,143.321762,132.972839,70.450714,83.969780
2024-05-28,90.176018,122.122604,40.839596,215.334763,174.374313,75.525421,90.824547,141.585083,133.241226,70.381744,83.456627
2024-05-29,88.887932,120.394165,40.512085,213.969620,173.229141,74.982422,89.220779,140.483521,132.535461,69.475250,82.677025


In [228]:
for row_idx,day in enumerate(time_period):
    # go through each ticker (across)
    for col_idx,ticker in enumerate(sector_etf_tickers):
        print(row_idx,col_idx)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
14 0
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
15 0
15 1
15 2
15 3
15 4
15 5
15 6
15 7
15 8
15 9
15 10
16 0
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
17 0
17 1
17 2
17 3
17 4
17 5
17 6
17 7
17 8
17 9
17 10
18 0
18 1
18 2
18 3
18 4
18 5
18 6
18 7
18 8
18 9
18 10
19 0
19 1
19 2
19 3
19 4
19 5
19 6
19 7
19 8
1

In [229]:
all_data['trough']['Adj Close']['expansion']

KeyError: 'Adj Close'

# Multiple Signals
After navigating Bollinger Bands, RSI, MACD and Ichimoku clouds with creating different signals, there is now a change to incorporate multiple signals into one investment. For each signals there is the amount that is bought/sold is dependent on the number of buy/sell signals. So if there are 4/4 signals then your invesment is going to 0.2 of your current balance.

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

def calculate_portfolio_roi(bb_signals_nd, adj_close_nd, periods_date, periods_list, tickers, n_sample, initial_investment):
    # Initialize a nested dictionary to store ROI percentages for each period and ticker
    roi_results = {period: [] for period in periods_list}

    # Loop through each economic period
    for period in periods_list:
        # Create the date range for the current period
        date_range = pd.date_range(start=pd.to_datetime(periods_date[period][0]), end=pd.to_datetime(periods_date[period][1]) - timedelta(days=90))
        
        # Get random dates for stochastic modeling
        start_dates = np.random.choice(date_range, size=n_sample, replace=True)

        # Loop through sampled start dates
        for start_date in start_dates:
            time_stamp = pd.to_datetime(start_date)

            # Initialize variables
            account_balance = initial_investment
            shares_number = {ticker: 0 for ticker in tickers}  # Initialize share count for each ticker

            # Extract the adjusted close and signal data for time period
            adj_close_period = adj_close_nd[period].loc[time_stamp:time_stamp + timedelta(days=90)]
            bb_signals_period = bb_signals_nd[period].loc[time_stamp:time_stamp + timedelta(days=90)]

            # Iterate over each row in the Bollinger Band signals (day by day)
            for row_idx, row in bb_signals_period.iterrows():
                # Calculate the number of signals for that day
                num_buy_signals = sum(signal == 'Buy' for signal in row)
                num_sell_signals = sum(signal == 'Sell' for signal in row)

                # Determine the total investment amount based on signals
                total_signals = num_buy_signals + num_sell_signals
                if total_signals > 0:
                    investment_per_signal = account_balance * (1 / total_signals)

                # Execute buy signals
                for col_idx, signal in enumerate(row):
                    ticker = tickers[col_idx]
                    adj_close_price = adj_close_period.loc[row_idx, ticker]

                    # Handle Buy action
                    if signal == 'Buy' and total_signals > 0:
                        amount_to_buy = investment_per_signal
                        if account_balance >= amount_to_buy:
                            shares_to_buy = amount_to_buy / adj_close_price
                            shares_number[ticker] += shares_to_buy
                            account_balance -= amount_to_buy

                    # Handle Sell action
                    elif signal == 'Sell':
                        if shares_number[ticker] > 0:
                            shares_value = shares_number[ticker] * adj_close_price
                            amount_to_sell = investment_per_signal
                            if shares_value >= amount_to_sell:
                                shares_to_sell = amount_to_sell / adj_close_price
                                shares_number[ticker] -= shares_to_sell
                                account_balance += amount_to_sell

            # Calculate total portfolio value at the end of the period
            portfolio_value = sum(shares_number[ticker] * adj_close_period.iloc[-1][ticker] for ticker in tickers)
            total_value = account_balance + portfolio_value
            
            # Calculate the profit relative to the initial investment
            profit = total_value - initial_investment
            
            # Store profit for this simulation
            roi_results[period].append(profit)

    return roi_results


In [ ]:
# create a function that is going to go through each day, then each ticker and then go through each signal for each stock for each day
